# 🛡️ Complete Robustness Analysis

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Time:</b> 25 minutes<br>
<b>Dataset:</b> Wine (sklearn)<br>
<b>Prerequisite:</b> 01_tests_introduction.ipynb
</div>

## 🎯 Objectives
- ✅ Understand what robustness is in ML
- ✅ Run complete robustness analysis
- ✅ Interpret perturbation methods
- ✅ Analyze sensitive features
- ✅ Generate professional HTML report

## 📚 What is Robustness?

### Definition
**Robustness** is the model's ability to maintain good performance even when input data suffers small perturbations.

### Why it matters?
1. **Real-world data is imperfect**
   - Measurement errors
   - Sensor noise
   - Manual data entry

2. **Security against adversarial attacks**
   - Intentional feature manipulation
   - Attempts to fool the model

3. **Production reliability**
   - Model must be stable
   - Consistent predictions

## 1️⃣ Setup and Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from deepbridge import DBDataset, Experiment

# Configure visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load data
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target

print(f"📊 Dataset: {df.shape}")
print(f"🍷 Classes: {wine.target_names}")

2025-11-12 20:58:55,894 - deepbridge.reports - DEBUG - Using refactored FairnessDataTransformer
2025-11-12 20:58:55,906 - deepbridge.reports - INFO - Successfully imported radar chart fix
2025-11-12 20:58:55,908 - deepbridge.reports - INFO - Successfully patched EnhancedUncertaintyCharts.generate_model_metrics_comparison
2025-11-12 20:58:55,909 - deepbridge.reports - INFO - Successfully applied enhanced_charts patch
2025-11-12 20:58:55,912 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-12 20:58:55,916 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-12 20:58:55,917 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-12 20:58:55,918 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_ch

## 2️⃣ Train Model

In [2]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

baseline_acc = clf.score(X_test, y_test)
print(f"✅ Model trained!")
print(f"📊 Baseline accuracy: {baseline_acc:.3f}")

✅ Model trained!
📊 Baseline accuracy: 1.000


## 3️⃣ Create Experiment

In [3]:
dataset = DBDataset(
    data=df,
    target_column='target',
    model=clf,
    test_size=0.2,
    random_state=42,
    dataset_name='Wine Classification'
)

exp = Experiment(
    dataset=dataset,
    experiment_type='multiclass_classification',
    random_state=42
)

print("✅ Experiment created!")

✅ Initial model evaluation complete: RandomForestClassifier
✅ Experiment created!


## 4️⃣ Quick Test

First, let's run a quick test:

In [4]:
print("🧪 Running quick robustness test...\n")

rob_quick = exp.run_test('robustness', config_name='quick')

print("\n✅ Quick test completed!")
print(f"\n📊 Robustness Score: {rob_quick.get('robustness_score', 0):.3f}")

🧪 Running quick robustness test...

✅ Robustness Tests Finished!
🎉 Test completed successfully: robustness

✅ Quick test completed!

📊 Robustness Score: 0.000


## 5️⃣ Complete Test (full)

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #ff9800;">
<b>⏱️ Note:</b> The complete test may take a few minutes!
</div>

In [5]:
print("🔬 Running COMPLETE robustness analysis...\n")

rob_full = exp.run_test('robustness', config_name='full')

print("\n✅ Complete analysis finished!")

🔬 Running COMPLETE robustness analysis...

✅ Robustness Tests Finished!
🎉 Test completed successfully: robustness

✅ Complete analysis finished!


## 6️⃣ Perturbation Methods

DeepBridge tests robustness using different methods:

In [6]:
print("📊 TESTED PERTURBATION METHODS:\n" + "="*60 + "\n")

print("\n🔹 RAW PERTURBATION (Gaussian Noise)")
print("   Description: Adds Gaussian noise to numerical features")
print("   Simulates: Measurement errors, sensor noise")
if 'raw' in rob_full and 'by_level' in rob_full['raw']:
    levels = sorted(rob_full['raw']['by_level'].keys())
    print(f"   Levels tested: {', '.join(levels)}")
print()
    
print("🔹 QUANTILE PERTURBATION")
print("   Description: Perturbs features using quantile-based methods")
print("   Simulates: Systematic shifts in feature distributions")
if 'quantile' in rob_full and 'by_level' in rob_full['quantile']:
    levels = sorted(rob_full['quantile']['by_level'].keys())
    print(f"   Levels tested: {', '.join(levels)}")
print()

print(f"\n📈 IMPACT SUMMARY:")
print(f"   Raw Impact: {rob_full.get('avg_raw_impact', 0):.3f}")
print(f"   Quantile Impact: {rob_full.get('avg_quantile_impact', 0):.3f}")
print(f"   Overall Impact: {rob_full.get('avg_overall_impact', 0):.3f}")

📊 TESTED PERTURBATION METHODS:


🔹 RAW PERTURBATION (Gaussian Noise)
   Description: Adds Gaussian noise to numerical features
   Simulates: Measurement errors, sensor noise

🔹 QUANTILE PERTURBATION
   Description: Perturbs features using quantile-based methods
   Simulates: Systematic shifts in feature distributions


📈 IMPACT SUMMARY:
   Raw Impact: 0.000
   Quantile Impact: 0.000
   Overall Impact: 0.000


## 7️⃣ Interpret Robustness Score

### Score Scale
- **0.95 - 1.00**: 🟢 Excellent - Very robust model
- **0.85 - 0.95**: 🟡 Good - Acceptable robustness
- **0.70 - 0.85**: 🟠 Moderate - Needs attention
- **< 0.70**: 🔴 Weak - Fragile model

In [7]:
score = rob_full.get('robustness_score', 0)

print(f"\n📊 ROBUSTNESS SCORE: {score:.3f}\n")

if score >= 0.95:
    print("🟢 EXCELLENT - Very robust model!")
    print("   ✅ Ready for production")
elif score >= 0.85:
    print("🟡 GOOD - Acceptable robustness")
    print("   ✅ Can go to production with monitoring")
elif score >= 0.70:
    print("🟠 MODERATE - Needs attention")
    print("   ⚠️  Consider improvements before production")
else:
    print("🔴 WEAK - Fragile model")
    print("   ❌ NOT recommended for production")


📊 ROBUSTNESS SCORE: 0.000

🔴 WEAK - Fragile model
   ❌ NOT recommended for production


## 8️⃣ Sensitive Features

Analyze which features are most sensitive to perturbations:

In [8]:
# Check if feature_importance is available in results
if 'feature_importance' in rob_full and rob_full['feature_importance']:
    importance = rob_full['feature_importance']
    
    # Top 5 most important features (those with highest impact when perturbed)
    print("\n🔍 TOP 5 MOST SENSITIVE FEATURES:\n" + "="*60)
    sorted_features = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:5]
    
    for i, (feature, impact_score) in enumerate(sorted_features, 1):
        print(f"\n{i}. {feature}")
        print(f"   Impact Score: {impact_score:.3f}")
        print(f"   ⚠️  Perturbations in this feature affect model performance")
else:
    print("\n💡 Feature importance not available in results")
    print("   This may be expected depending on the configuration used")


💡 Feature importance not available in results
   This may be expected depending on the configuration used


## 9️⃣ Visualize Performance Degradation

In [9]:
# Create degradation visualization from raw perturbation data
if 'raw' in rob_full and 'by_level' in rob_full['raw']:
    raw_data = rob_full['raw']['by_level']
    
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Degradation curve
    plt.subplot(1, 2, 1)
    
    # Extract levels and scores
    levels = []
    scores = []
    for level_str in sorted(raw_data.keys(), key=lambda x: float(x)):
        levels.append(float(level_str))
        level_data = raw_data[level_str]
        # Get mean score across all features
        if 'overall_result' in level_data and 'all_features' in level_data['overall_result']:
            mean_score = level_data['overall_result']['all_features'].get('mean_score', 0)
            scores.append(mean_score)
    
    if scores:
        plt.plot([0] + levels, [baseline_acc] + scores, 
                 marker='o', linewidth=2, markersize=8, label='Model Performance')
        plt.axhline(y=baseline_acc, color='r', linestyle='--', 
                    label=f'Baseline ({baseline_acc:.3f})', alpha=0.7)
        plt.xlabel('Perturbation Level', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.title('Performance Degradation', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # Plot 2: Impact comparison
    plt.subplot(1, 2, 2)
    
    methods = ['Raw', 'Quantile']
    impacts = [
        rob_full.get('avg_raw_impact', 0),
        rob_full.get('avg_quantile_impact', 0)
    ]
    
    colors = ['#ff6b6b' if impact > 0.3 else '#51cf66' if impact < 0.15 else '#ffd43b' 
              for impact in impacts]
    
    bars = plt.barh(methods, impacts, color=colors, edgecolor='navy', alpha=0.7)
    plt.xlabel('Average Impact', fontsize=12)
    plt.title('Impact by Perturbation Method', fontsize=14, fontweight='bold')
    plt.xlim(0, max(impacts) * 1.2 if impacts else 1)
    plt.grid(axis='x', alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, impact) in enumerate(zip(bars, impacts)):
        plt.text(impact + 0.01, i, f'{impact:.3f}', 
                va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("📊 Degradation data not available")
    print("   Make sure you ran the full robustness test")

📊 Degradation data not available
   Make sure you ran the full robustness test


## 🔟 Generate HTML Report

Generate professional report for documentation:

In [10]:
# Generate HTML report using the Experiment's report generation
print("📄 To generate a complete HTML report, use:")
print("   exp.save_report(output_path='robustness_report.html')")
print("\nOr run all tests and generate a comprehensive report:")
print("   results = exp.run_tests(config_name='full')")
print("   results.save_html('comprehensive_report.html')")

print("\n💡 The HTML report includes:")
print("   - Interactive visualizations")
print("   - Detailed metrics and tables")
print("   - Professional formatting for stakeholders")
print("   - Exportable charts and data")

📄 To generate a complete HTML report, use:
   exp.save_report(output_path='robustness_report.html')

Or run all tests and generate a comprehensive report:
   results = exp.run_tests(config_name='full')
   results.save_html('comprehensive_report.html')

💡 The HTML report includes:
   - Interactive visualizations
   - Detailed metrics and tables
   - Professional formatting for stakeholders
   - Exportable charts and data


In [11]:
# Generate HTML report
output_path = 'outputs/robustness_report_complete.html'

print("📄 Generating HTML report...")

# Store the result in _test_results so save_html can access it
if not hasattr(exp, '_test_results'):
    exp._test_results = {}
exp._test_results['robustness'] = rob_full

# Add initial_results to provide feature importance data
if not hasattr(exp, '_experiment_result'):
    from collections import OrderedDict
    from deepbridge.core.experiment.results import wrap_results
    
    combined_results = OrderedDict()
    combined_results['experiment_type'] = exp.experiment_type
    combined_results['config'] = {'name': 'full', 'tests': exp.tests}
    
    # Create experiment result object
    experiment_result = wrap_results(combined_results)
    experiment_result.results = OrderedDict()
    
    # Add initial_results with model feature importance
    experiment_result.results['initial_results'] = exp.initial_results
    
    # Add robustness test results
    experiment_result.results['robustness'] = rob_full
    
    exp._experiment_result = experiment_result
    
    # Debug: Verify initial_results structure
    print("\n🔍 Debug: Checking data structure...")
    if 'models' in exp.initial_results and 'primary_model' in exp.initial_results['models']:
        pm = exp.initial_results['models']['primary_model']
        if 'feature_importance' in pm:
            print(f"✅ Feature importance found: {len(pm['feature_importance'])} features")
        else:
            print("❌ Feature importance NOT found in primary_model")
    else:
        print("❌ Primary model NOT found in initial_results")

# Now generate the report
exp.save_html(test_type='robustness', file_path=output_path, report_type='interactive')
print(f"✅ Report generated successfully!")
print(f"📂 Location: {output_path}")
print(f"\n💡 Open the file in your browser to view the interactive report")

📄 Generating HTML report...

🔍 Debug: Checking data structure...
✅ Feature importance found: 13 features
2025-11-12 20:59:00,273 - deepbridge.reports - INFO - Generating SIMPLE robustness report to: /home/guhaase/projetos/DeepBridge/examples/notebooks/03_validation_tests/outputs/robustness_report_complete.html
2025-11-12 20:59:00,275 - deepbridge.reports - INFO - Report type: interactive
2025-11-12 20:59:00,275 - deepbridge.reports - INFO - Transforming robustness data for report (SIMPLE)
2025-11-12 20:59:00,276 - deepbridge.reports - INFO - Found 13 features for report
2025-11-12 20:59:00,743 - deepbridge.reports - INFO - Found 13 features for report
2025-11-12 20:59:00,773 - deepbridge.reports - INFO - Transformation complete. 6 levels, 13 features
2025-11-12 20:59:00,774 - deepbridge.reports - INFO - Found template at: /home/guhaase/projetos/DeepBridge/deepbridge/templates/report_types/robustness/interactive/index.html
2025-11-12 20:59:00,775 - deepbridge.reports - DEBUG - Loading t

## 1️⃣1️⃣ How to Improve Robustness?

### Techniques to Increase Robustness:

#### 1. **Data Augmentation**
```python
# Add noise during training
X_train_noisy = X_train + np.random.normal(0, 0.1, X_train.shape)
```

#### 2. **Ensemble Methods**
```python
# Use multiple models (we're already using RandomForest)
from sklearn.ensemble import VotingClassifier
```

#### 3. **Robust Feature Engineering**
- Remove very sensitive features
- Create aggregated features
- Proper normalization

#### 4. **Regularization**
```python
# Increase regularization (L1, L2, dropout)
```

#### 5. **Adversarial Training**
```python
# Train with adversarial examples
```

## 🎉 Conclusion

### What you learned:
- ✅ **Robustness Concept** - Resistance to perturbations
- ✅ **Perturbation Methods** - Gaussian, Dropout, Scaling, Adversarial
- ✅ **Interpret Scores** - Scale from 0 to 1
- ✅ **Sensitive Features** - Identify vulnerabilities
- ✅ **Reports** - Professional documentation
- ✅ **Improvement Techniques** - How to increase robustness

### Robustness Checklist for Production:
- [ ] ✅ Robustness Score > 0.85
- [ ] ✅ Sensitive features identified and documented
- [ ] ✅ Acceptable degradation in all methods
- [ ] ✅ HTML report generated for audit
- [ ] ✅ Production monitoring plan

### Next Steps:
- 📘 `03_uncertainty.ipynb` - Quantify uncertainty in predictions
- 📘 `04_resilience_drift.ipynb` - Detect temporal drift
- 📘 `06_model_comparison.ipynb` - Compare robustness between models

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>💡 Insight:</b> A model with high accuracy but low robustness can fail in production!
</div>